In [1]:
import numpy as np
import pandas as pd

from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer

### Import Data

In [2]:
df = pd.read_csv("train_data.csv")
df['kata'] = df['kata'].values.astype('U')

In [4]:
df['kata'].head()

0         grownup
1    denotational
2           kribo
3    melagukannya
4       wrangling
Name: kata, dtype: object

In [13]:
df.bahasa.value_counts()

indonesia    58354
inggris      54050
Name: bahasa, dtype: int64

In [14]:
# indonesia = df[df.bahasa=="indonesia"]
# indonesia.head()

In [15]:
# inggris = df[df.bahasa=="inggris"]
# inggris.head()

### Split Dataset

In [16]:
X_train, X_test, y_train, y_test = train_test_split(df["kata"], df["bahasa"], test_size=0.1, random_state=123)

### Trigraph

In [17]:
cvec = CountVectorizer(max_df=1.0, min_df=1, analyzer='char', ngram_range=(1,3))
tfidf_trans = TfidfTransformer()
text_clf = Pipeline([
    ('vect', cvec),
    ('tfidf',tfidf_trans),
    ('clf', MLPClassifier(activation="logistic",hidden_layer_sizes=20)),
])

text_clf.fit(X_train, y_train)

print(text_clf.score(X_train, y_train))

0.9720352302719374


In [18]:
print(text_clf.score(X_test, y_test))

0.9584556534116182


In [19]:
text_clf.predict(["chair"])

array(['inggris'], dtype='<U9')

In [50]:
##Development
pred = text_clf.predict(X_test)
akurasi = np.mean(pred==y_test)
print("Akurasi: {}".format(akurasi))

print(classification_report(y_test, pred))

Akurasi: 0.9584556534116182
              precision    recall  f1-score   support

   indonesia       0.96      0.97      0.96      5835
     inggris       0.96      0.95      0.96      5406

   micro avg       0.96      0.96      0.96     11241
   macro avg       0.96      0.96      0.96     11241
weighted avg       0.96      0.96      0.96     11241



In [36]:
def identify_language(kata):
    search = df.loc[df['kata'].isin([kata])]
    if ('indonesia') in search.bahasa.values:
        skor_indonesia = 1
    else:
        skor_indonesia = 0.1
    if ('inggris') in search.bahasa.values:
        skor_inggris = 1
    else: 
        skor_inggris = 0.1
    skor_ngraph=text_clf.predict_proba([kata])
    skor_indonesia = skor_indonesia * skor_ngraph[0][0]
    skor_inggris = skor_inggris * skor_ngraph[0][1]
    
    if (skor_indonesia >= skor_inggris):
        return "indonesia"
    else:
        return "inggris"

In [37]:
testword = "salahku"

In [38]:
identify_language(testword)

'indonesia'

In [39]:
predict = []
for kata in X_test:
#     print(kata)
    predict.append(identify_language(kata))

In [40]:
akurasi = np.mean(predict==y_test)
print("Akurasi: {}".format(akurasi))

print(classification_report(y_test, predict))

Akurasi: 0.9797171070189485
              precision    recall  f1-score   support

   indonesia       0.98      0.98      0.98      5835
     inggris       0.98      0.97      0.98      5406

   micro avg       0.98      0.98      0.98     11241
   macro avg       0.98      0.98      0.98     11241
weighted avg       0.98      0.98      0.98     11241



In [41]:
predict

['indonesia',
 'inggris',
 'indonesia',
 'inggris',
 'inggris',
 'indonesia',
 'inggris',
 'inggris',
 'inggris',
 'indonesia',
 'inggris',
 'indonesia',
 'indonesia',
 'indonesia',
 'indonesia',
 'inggris',
 'inggris',
 'indonesia',
 'inggris',
 'indonesia',
 'indonesia',
 'inggris',
 'indonesia',
 'indonesia',
 'inggris',
 'indonesia',
 'inggris',
 'indonesia',
 'indonesia',
 'indonesia',
 'inggris',
 'inggris',
 'indonesia',
 'indonesia',
 'inggris',
 'inggris',
 'indonesia',
 'inggris',
 'inggris',
 'indonesia',
 'indonesia',
 'indonesia',
 'indonesia',
 'inggris',
 'indonesia',
 'inggris',
 'inggris',
 'indonesia',
 'indonesia',
 'indonesia',
 'inggris',
 'indonesia',
 'inggris',
 'indonesia',
 'inggris',
 'inggris',
 'indonesia',
 'indonesia',
 'inggris',
 'indonesia',
 'indonesia',
 'inggris',
 'inggris',
 'inggris',
 'inggris',
 'indonesia',
 'indonesia',
 'inggris',
 'inggris',
 'indonesia',
 'inggris',
 'inggris',
 'indonesia',
 'indonesia',
 'inggris',
 'indonesia',
 'inggri

In [42]:
# TEST
df_test = pd.read_csv("test_data.csv")
df_test['kata'] = df_test['kata'].values.astype('U')

In [43]:
predict_test = []
for kata in df_test['kata']:
    predict_test.append(identify_language(kata))

In [45]:
print(text_clf.score(df_test['kata'], df_test['bahasa']))

0.9606885508406725


In [44]:
akurasi = np.mean(predict_test==df_test['bahasa'])
print("Akurasi: {}".format(akurasi))

print(classification_report(df_test['bahasa'],predict_test))

Akurasi: 0.9562049639711769
              precision    recall  f1-score   support

   indonesia       0.96      0.96      0.96      6417
     inggris       0.95      0.96      0.95      6073

   micro avg       0.96      0.96      0.96     12490
   macro avg       0.96      0.96      0.96     12490
weighted avg       0.96      0.96      0.96     12490

